# FLARE Logging

This tutorial covers how to configure logging in FLARE for different use cases and modes.

To learn more, see our [Logging Configuration Documentation](https://nvflare.readthedocs.io/en/main/user_guide/configurations/logging_configuration.html) for a more comprehensive description of the various features.

## Setup

The NVFlare [Quickstart Guide](https://nvflare.readthedocs.io/en/main/quickstart.html#installation) provides instructions for setting up FLARE on a local system or in a Docker image.  We've also cloned the NVFlare GitHub in our top-level working directory.

## Simulator Logging

To get started, let's run the **hello-numpy-sag** job in the simulator and take a look at the default logging output.

The simulator `--log_config` (`-l`) argument can be used to set the log config mode ('concise', 'full', 'verbose'), filepath to a log config json file, or level (info, debug, error, etc.).
For this example we first show the `"full"` mode to compare it with our custom configuration.

Note: this argument is defaulted to `"concise"` in the simulator mode, however feel free to experiment with the other modes or levels.

In [ ]:
!mkdir -p hello-numpy-sag-workspace
!nvflare simulator -w hello-numpy-sag-workspace -n 2 -t 2 -l full ../hello-world/hello-numpy-sag/jobs/hello-numpy-sag 

Notice how the output contains lots of logs from both the FLARE system, as well as the training workflow.
Additionally, the different level of logs (eg. INFO, WARNING, ERROR) have different console colors.

We now will cover custom log configurations using the log config json file option. First lets look at the default configuration used in this run and the generated log files in the workspace:

In [ ]:
!tree hello-numpy-sag-workspace

### Default Log Config

The **log_config.json** is the default logging configuration used. 
This configuration comes with pre-configured handlers for console level colors, logs, error logs, structured json logs, and fl training logs using the following main sections:

- formatters: formatting the structure of the log records
- filters: filters the log based on a specified criteria
- handler: sends logs to a destination, can use formatter and filters
- loggers: configure root logger with handlers, and any other loggers

Let's take a look at the configuration under the server (will be the same for all sites by default):

In [ ]:
!cat hello-numpy-sag-workspace/server/local/log_config.json

### Default Log Files

Next we can look at the various log files that are created by the FileHandlers.
Below we look at the server logs, but feel free to also check out the site logs as well.

#### log.txt

The logFileHandler uses the baseFormatter to write all logs to log.txt.
This is the default log that we see in the console:

In [ ]:
!cat hello-numpy-sag-workspace/server/log.txt

#### log.json

The jsonFileHandler uses the jsonFormatter to write json formatted logs to log.json.
This is useful for leveraging the structured logs (ie with a 3rd party observability package):

In [ ]:
!cat hello-numpy-sag-workspace/server/log.json

#### log_error.txt

The errorFileHandler uses the baseFormatter and level “ERROR” to write error level logs to log_error.txt.
This allows users to easily see when errors are logged:

In [6]:
!cat hello-numpy-sag-workspace/server/log_error.txt

#### log_fl.txt

The FLFileHandler uses the baseFormatter and FLFilter (uses LoggerNameFilter allowing certain logger names) to write fl training and custom logs to log_fl.txt.
This removes the system and communication related logs and clearly shows logs related to FL training:

In [ ]:
!cat hello-numpy-sag-workspace/server/log_fl.txt

### Customization

The log config file can easily be customized for different use cases.

For this example, let's assume we are a federated learning researcher mainly interested in the algorithm parts of the log.
By default, we provide the FLFileHandler to generate the log_fl.txt However say we would also like to have a cleaner output in the console, as well as reduce any unnecessary log fields.

#### custom_log_config.json

In custom_log_config.json, let's see how we modify the consoleHandler and consoleFormatter to achieve our goal of a cleaner console output.
Below are some example changes, however feel free to experiment with different configurations:

**Filters:** In the consoleHandler, we add the FLFilter which only allows logs related to FL training to pass through using the the LoggerNameFilter:

- ``"filters": ["FLFilter"],``

**Log Format:** In the consoleFormatter, we remove the fl_ctx field and add the identity field for a cleaner log structure:

- `"fmt": "%(asctime)s - %(identity)s - %(name)s - %(levelname)s - %(message)s",`

**Log Date Format:** In the consoleFormatter, we configure the datefmt to only use seconds rather than milliseconds:

- ``"datefmt": "%Y-%m-%d %H:%M:%S"``

**Color Format**: In the consoleFormatter, to highlight the NPModelPersistor for example, we can color it blue to make it stand out in the console:
- ```
  "logger_colors": {
      "NPModelPersistor": "blue"
  }
  ```

**Logger Hierarchy**: In the consoleFormatter, to turn all loggers under nvflare.app_common.aggregators to DEBUG level for example, we can configure it under loggers (note: FLARE loggers are organized matching package hierarchy with dot separated name, allowing for configuration at different granularities. Additionally, logs from children loggers will by default propagate up to parent loggers and their handlers):
- ```
  "nvflare.app_common.aggregrators": {
      "level": "DEBUG"
  }
  ```



In [ ]:
!cat custom_log_config.json

In [ ]:
!diff custom_log_config.json hello-numpy-sag-workspace/server/local/log_config.json

Now let's run the simulator with the custom_log_config.json using the ``-l`` option:

In [ ]:
!nvflare simulator -w hello-numpy-sag-workspace -n 2 -t 2 -l custom_log_config.json ../hello-world/hello-numpy-sag/jobs/hello-numpy-sag

Compare this to the original output from the first command, and note the differences in the log output. Note that this concise format can also be achieved using the simulator "concise" mode, however this helps cover how to customize the logs using the file.

In addition to the consoleHandler, all the other formatters, filters, handlers, and loggers can all also be customized just as easily.

See the [Logging Configuration Documentation](https://nvflare.readthedocs.io/en/main/user_guide/configurations/logging_configuration.html) for more information on how to customize the different sections.

## Provisioned System and Logging Configuration Commands

For this part of the example, we will showcase how to use the [Dynamic Logging Configuration Commands](https://nvflare.readthedocs.io/en/main/user_guide/configurations/logging_configuration.html#dynamic-logging-configuration-commands) with a running FLARE system.

To provision and start an FL system, you can use [POC mode](setup_poc.ipynb) to quickly get started. Feel free to use an existing **provisioned** FLARE project if you have that available. Remember that we recommend starting the system in a separate terminal. 

Once the system is running and you have logged into the admin console you are ready to try out the commands. We provide two admin commands to enable users to dynamically configure the site or job level logging:

### configure_site_log

Configures the site level logs, but does not affect the job logs.

Usage: ``configure_site_log <target> <config>``

- **target**: server, client <clients>..., or all
- **config**: log configuration
    - log mode (concise, full, verbose)
    - path to a json log configuration file (/path/to/my_log_config.json)
    - log level name/number (debug, INFO, 30)
    - read the current log configuration file (reload)

Try and experiment with the following commands in the admin console:

- ``configure_site_log server debug``
- ``configure_site_log client site-1 debug``
- ``configure_site_log all info``

### configure_job_log

Configures the job logs, does not affect site logs.

Usage: ``configure_job_log <job_id> <target> <config>``

- **job_id**: id of a running job
- **target**: server, client <clients>..., or all
- **config**: log configuration (see above)

Submit a job with ``submit_job <path>/<to>/<job_folder>``,
then try and experiment with the following commands in the admin console:


- ``configure_job_log <job_id> server debug``
- ``configure_job_log <job_id> client site-1 debug``
- ``configure_job_log <job_id> all info``
- ``configure_job_log <job_id> all <path>/<to>/custom_log_config.json``

Lastly, take a look at the generated log files in the workspace, noting the difference between the site and job logs.
For example if using POC mode:

In [ ]:
!tree /tmp/nvflare/poc

## Defining Loggers in the Hierarchy

When defining new loggers, we provide several functions to help adhere to the FLARE package logger hierarchy. For example say we have the following module at `my_package.my_module`:

- [get_obj_logger](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.fuel.utils.log_utils.html#nvflare.fuel.utils.log_utils.get_obj_logger) for classes. Ex: 
```
        class MyClass:
            def __init__(self):
            self.logger = get_obj_logger(self) # my_package.my_module.MyClass
```

- [get_script_logger](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.fuel.utils.log_utils.html#nvflare.fuel.utils.log_utils.get_script_logger) for scripts (if not in a package, default to custom.<script_file_name>). Ex:
```
        if __name__ == "__main__":
            logger = get_script_logger() # my_package.my_module
```

- [get_module_logger](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.fuel.utils.log_utils.html#nvflare.fuel.utils.log_utils.get_module_logger) for modules. Ex:
```
        def my_function():
            logger = get_module_logger(name="my_function") # my_package.my_module.my_function
```


For more information, refer to the [Logging Configuration Documentation](https://nvflare.readthedocs.io/en/main/user_guide/configurations/logging_configuration.html#loggers) for definining loggers. 